# Environment

In [2]:
## use instance ml.m5.large for SentenceTransformers
!pip install -U sentence-transformers
!pip install kneed

In [2]:
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt

from functools import reduce
# from kneed import KneeLocator
from sklearn.cluster import KMeans
from collections import defaultdict
from plotly.subplots import make_subplots
from sklearn.metrics import silhouette_score
# from sentence_transformers import SentenceTransformer

%matplotlib inline 
pio.renderers.default='iframe'
warnings.filterwarnings(action = 'ignore')

# Setup

In [5]:
def setup():
    file_path = "Top 25 skills for Sub Analysis (By Year and ONET | with NAICS filter).xlsx"
    new_columns = ['onet_parent','onet_child']
    df_1 = pd.read_excel(file_path, sheet_name='2010')
    df_1[new_columns] = df_1['onet'].str.split('-', 1, expand=True)
    df_2 = pd.read_excel(file_path, sheet_name='2011')
    df_2[new_columns] = df_2['onet'].str.split('-', 1, expand=True)
    df_3 = pd.read_excel(file_path, sheet_name='2012')
    df_3[new_columns] = df_3['onet'].str.split('-', 1, expand=True)
    df_4 = pd.read_excel(file_path, sheet_name='2013')
    df_4[new_columns] = df_4['onet'].str.split('-', 1, expand=True)
    df_5 = pd.read_excel(file_path, sheet_name='2014')
    df_5[new_columns] = df_5['onet'].str.split('-', 1, expand=True)
    df_6 = pd.read_excel(file_path, sheet_name='2015')
    df_6[new_columns] = df_6['onet'].str.split('-', 1, expand=True)
    df_7 = pd.read_excel(file_path, sheet_name='2016')
    df_7[new_columns] = df_7['onet'].str.split('-', 1, expand=True)
    df_8 = pd.read_excel(file_path, sheet_name='2017')
    df_8[new_columns] = df_8['onet'].str.split('-', 1, expand=True)
    df_9 = pd.read_excel(file_path, sheet_name='2018')
    df_9[new_columns] = df_9['onet'].str.split('-', 1, expand=True)
    df_10 = pd.read_excel(file_path, sheet_name='2019')
    df_10[new_columns] = df_10['onet'].str.split('-', 1, expand=True)
    df_11 = pd.read_excel(file_path, sheet_name='2020')
    df_11[new_columns] = df_11['onet'].str.split('-', 1, expand=True)
    df_12 = pd.read_excel(file_path, sheet_name='2021')
    df_12[new_columns] = df_12['onet'].str.split('-', 1, expand=True)
    return df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12

In [6]:
df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021 = setup()
dfs = [df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021]

years = list(range(2010, 2022))
yearly_dfs = {2010: df_2010, 2011: df_2011, 2012: df_2012, 2013: df_2013, 2014: df_2014, 2015: df_2015, 
              2016: df_2016, 2017: df_2017, 2018: df_2018, 2019: df_2019, 2020: df_2020, 2021: df_2021}

# ONET Codes

In [7]:
!rm -rf 'Sub Analysis'
!mkdir 'Sub Analysis'

In [3]:
K = range(2,30)
parent_folder = 'Sub Analysis'
title = 'Sub Analysis for the News Media ONET Codes'
onet_model = parent_folder+'/model.pkl'
# language_model = 'all-MiniLM-L6-v2'
language_model = 'all-mpnet-base-v2'
onet_k_value = parent_folder+'/k_value.png'
onet_cluster_data = parent_folder+'/cluster_data'
onet_bar_chart = parent_folder+'/bar_charts.html'
onet_line_chart = parent_folder+'/line_chart.html'
onet_bubble_chart = parent_folder+'/bubble_chart.html'
normalized_onet_cluster_data = parent_folder+'/normalized_cluster_data'
normalized_onet_bar_chart = parent_folder+'/normalized_bar_charts.html'
normalized_onet_line_chart = parent_folder+'/normalized_line_chart.html'
normalized_onet_bubble_chart = parent_folder+'/normalized_bubble_chart.html'

## Create Embeddings

In [8]:
## Generate list of skills
skills = []
skills_per_year = []
for df in dfs:
    skills.extend(df['skill'].unique().tolist())
    skills_per_year.append(df['skill'].unique().tolist())
skills = list(set(skills))

In [9]:
## Create Embeddings
model = SentenceTransformer(language_model)
skills_embeddings = model.encode(skills)
skills_per_year_embeddings = []
for item in skills_per_year:
    skills_per_year_embeddings.append(model.encode(item))

## k-means clustering

### Fit a Model

In [10]:
# using elbow method to figure out ideal number of clusters
# distortions = []
score=0
for k in K:
    model = KMeans(n_clusters=k, init='k-means++')
    model.fit_predict(skills_embeddings)
#     distortions.append(model.inertia_)
    new_score = silhouette_score(skills_embeddings, model.labels_, metric='euclidean')
    if new_score>score:
        number_of_clusters = k
        score=new_score

print("Ideal K value based on silhouette score is {} with a score of {}".format(number_of_clusters, score))
# kn = KneeLocator(list(K), distortions, S=1.0, curve='convex', direction='decreasing')
# number_of_clusters = kn.knee
# print("Ideal K value based on elbow method = {}".format(number_of_clusters))
# plt.xlabel('k')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method showing the optimal k')
# plt.plot(K, distortions, 'bx-')
# plt.vlines(number_of_clusters, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
# plt.savefig(onet_k_value, bbox_inches='tight')

Ideal K value based on silhouette score is 26 with a score of 0.049083661288022995


### Metrics

In [11]:
model = KMeans(n_clusters=number_of_clusters)
cluster_assignment = model.fit_predict(skills_embeddings)

clustered_skills = {}
for skill_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_skills:
        clustered_skills[cluster_id] = []
    clustered_skills[cluster_id].append(skills[skill_id])

score = silhouette_score(skills_embeddings, model.labels_, metric='euclidean')
print('Silhouette Score: %.3f' % score)

Silhouette Score: 0.048


### Save Output

In [12]:
cluster_data = "{'silhouette_score':'"+str(score)+"',"
for key in sorted(clustered_skills):
    cluster_data=cluster_data+str(key)+":"+str(clustered_skills[key])+","
cluster_data=cluster_data[:-1]+"}"
with open(onet_cluster_data+".json", 'w') as f:
    f.write(cluster_data)

In [13]:
with open(onet_model, "wb") as f:
    pickle.dump(model, f)
with open(onet_model, "rb") as f:
    model = pickle.load(f)

### Predict each year individually

In [14]:
df = pd.DataFrame(columns=['year', 'skills', 'cluster', 
                           'cluster_centers', 'cluster_centers_x', 'cluster_centers_y', 
                           'cluster_skill_count'], dtype=object)

for i in range(0, len(skills_per_year_embeddings)):
    year=years[i]
    print("Predicting for year {}".format(year))
    cluster_assignment = model.predict(skills_per_year_embeddings[i])

    clustered_skills = {}
    for skill_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id not in clustered_skills:
            clustered_skills[cluster_id] = []
        clustered_skills[cluster_id].append(skills_per_year[i][skill_id])
    
    for key in sorted(clustered_skills):
        count = 0
        for skill in clustered_skills[key]:
            count += yearly_dfs[year].loc[yearly_dfs[year]['skill'] == skill, 'count'].sum()
        row = {'year': year, 'skills':', '.join(clustered_skills[key]), 'cluster':key, 
               'cluster_centers': key+1,'cluster_centers_x': model.cluster_centers_[:,0],
               'cluster_centers_y': model.cluster_centers_[:,1], 'cluster_skill_count':count}
        df = df.append(row, ignore_index=True)
        
df.year = df.year.astype(int)
df.cluster = df.cluster.astype(int)
df.cluster_skill_count = df.cluster_skill_count.astype(int)

Predicting for year 2010
Predicting for year 2011
Predicting for year 2012
Predicting for year 2013
Predicting for year 2014
Predicting for year 2015
Predicting for year 2016
Predicting for year 2017
Predicting for year 2018
Predicting for year 2019
Predicting for year 2020
Predicting for year 2021


In [15]:
df.to_csv(onet_cluster_data+".csv")

### Visualizations

In [16]:
df = pd.read_csv(onet_cluster_data+".csv")

In [17]:
fig = px.scatter(
    df,
    x='cluster_centers',
    y='cluster_centers',
    animation_frame='year',
    animation_group='cluster',
    size='cluster_skill_count',
    color='cluster',
    hover_name="cluster",
    hover_data={ 'year':False, 'cluster_centers':False, 'cluster':False, 'skills': True},
    labels={'cluster_skill_count': 'Count', 'skills': 'Skills', 'cluster_centers': "Cluster"},
    log_x=False,
    range_x=[0, number_of_clusters+1],
    range_y=[0, number_of_clusters+1],
    title=title
)
fig.update(layout_coloraxis_showscale=True)
# fig.show()
fig.write_html(onet_bubble_chart)

In [18]:
labels={'cluster_skill_count': 'Count', 'cluster': "Cluster"}
figures = [
    px.bar(df[df['year']==2010], x='cluster', y='cluster_skill_count', title='2010', labels=labels),
    px.bar(df[df['year']==2011], x='cluster', y='cluster_skill_count', title='2011', labels=labels),
    px.bar(df[df['year']==2012], x='cluster', y='cluster_skill_count', title='2012', labels=labels),
    px.bar(df[df['year']==2013], x='cluster', y='cluster_skill_count', title='2013', labels=labels),
    px.bar(df[df['year']==2014], x='cluster', y='cluster_skill_count', title='2014', labels=labels),
    px.bar(df[df['year']==2015], x='cluster', y='cluster_skill_count', title='2015', labels=labels),
    px.bar(df[df['year']==2016], x='cluster', y='cluster_skill_count', title='2016', labels=labels),
    px.bar(df[df['year']==2017], x='cluster', y='cluster_skill_count', title='2017', labels=labels),
    px.bar(df[df['year']==2018], x='cluster', y='cluster_skill_count', title='2018', labels=labels),
    px.bar(df[df['year']==2019], x='cluster', y='cluster_skill_count', title='2019', labels=labels),
    px.bar(df[df['year']==2020], x='cluster', y='cluster_skill_count', title='2020', labels=labels),
    px.bar(df[df['year']==2021], x='cluster', y='cluster_skill_count', title='2021', labels=labels)
]

fig = make_subplots(rows=6, cols=2, subplot_titles=range(2010, 2022)) 
        
for i, figure in enumerate(figures):
    if i==0:
        r, c = 1, 1
    elif i==1:
        r, c = 1, 2
    elif i==2:
        r, c = 2, 1
    elif i==3:
        r, c = 2, 2
    elif i==4:
        r, c = 3, 1
    elif i==5:
        r, c = 3, 2
    elif i==6:
        r, c = 4, 1
    elif i==7:
        r, c = 4, 2
    elif i==8:
        r, c = 5, 1
    elif i==9:
        r, c = 5, 2
    elif i==10:
        r, c = 6, 1
    elif i==11:
        r, c = 6, 2
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=r, col=c)

fig.update_layout(height=1500, width=1500, title_text=title)
fig.update_xaxes(tickmode='linear')
# fig.show()
fig.write_html(onet_bar_chart)

In [19]:
labels={'cluster_skill_count': 'Count', 'cluster': "Cluster"}

fig = px.line(df, x='year', y='cluster_skill_count', color='cluster', labels=labels)

fig.update_layout(height=500, width=1500, title_text=title)
fig.update_xaxes(tickmode='linear')
fig.write_html(onet_line_chart)

> ***Now that we are done with a basic comparison, let us normalize the data and try to visualize it.***

### Normalization

In [4]:
df = pd.read_csv(onet_cluster_data+".csv", index_col=0)

In [5]:
display(df.head(1))

,year,skills,cluster,cluster_centers,cluster_centers_x,cluster_centers_y,cluster_skill_count
0,2010,"Mainstreaming, Self-Starter, Sociology",0,1,[ 0.00337401 0.03282966 0.0326455 0.006920...,[ 0.03909327 -0.01626172 0.00322638 -0.030199...,412


In [6]:
new_df = pd.DataFrame()
clusters = df['cluster'].unique().tolist()
for cluster in clusters:
    temp_df = df[df['cluster']==cluster]
    min_val = temp_df['cluster_skill_count'].min()
    max_val = temp_df['cluster_skill_count'].max()
    temp_df['normalized_score'] = (temp_df['cluster_skill_count'] - min_val)/(max_val-min_val)
    new_df = pd.concat([new_df,temp_df])

In [16]:
new_df.to_csv(normalized_onet_cluster_data+".csv")

In [9]:
df = pd.read_csv(normalized_onet_cluster_data+".csv", index_col=0)

In [10]:
fig = px.scatter(
    df,
    x='cluster_centers',
    y='cluster_centers',
    animation_frame='year',
    animation_group='cluster',
    size='normalized_score',
    color='cluster',
    hover_name="cluster",
    hover_data={ 'year':False, 'cluster_centers':False, 'cluster':False, 'skills': True, 'normalized_score': True, 'cluster_skill_count': True},
    labels={'cluster_skill_count': 'Count', 'skills': 'Skills', 'cluster_centers': "Cluster", 'normalized_score': 'Normalized Score'},
    log_x=False,
    range_x=[0, number_of_clusters+1],
    range_y=[0, number_of_clusters+1],
    title=title
)
fig.update(layout_coloraxis_showscale=True)
# fig.show()
fig.write_html(normalized_onet_bubble_chart)

In [11]:
figures = [
    px.bar(df[df['year']==2010], x='cluster', y='normalized_score', title='2010', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2011], x='cluster', y='normalized_score', title='2011', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2012], x='cluster', y='normalized_score', title='2012', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2013], x='cluster', y='normalized_score', title='2013', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2014], x='cluster', y='normalized_score', title='2014', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2015], x='cluster', y='normalized_score', title='2015', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2016], x='cluster', y='normalized_score', title='2016', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2017], x='cluster', y='normalized_score', title='2017', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2018], x='cluster', y='normalized_score', title='2018', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2019], x='cluster', y='normalized_score', title='2019', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2020], x='cluster', y='normalized_score', title='2020', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    ),
    px.bar(df[df['year']==2021], x='cluster', y='normalized_score', title='2021', custom_data=['cluster_skill_count'])
    .update_traces(
        hovertemplate="<br>".join([
            "Cluster: %{x}",
            "Count: %{customdata[0]}",
            "Normalized Score: %{y}"
        ])
    )
]

fig = make_subplots(rows=6, cols=2, subplot_titles=range(2010, 2022)) 
        
for i, figure in enumerate(figures):
    if i==0:
        r, c = 1, 1
    elif i==1:
        r, c = 1, 2
    elif i==2:
        r, c = 2, 1
    elif i==3:
        r, c = 2, 2
    elif i==4:
        r, c = 3, 1
    elif i==5:
        r, c = 3, 2
    elif i==6:
        r, c = 4, 1
    elif i==7:
        r, c = 4, 2
    elif i==8:
        r, c = 5, 1
    elif i==9:
        r, c = 5, 2
    elif i==10:
        r, c = 6, 1
    elif i==11:
        r, c = 6, 2
    for trace in range(len(figure["data"])):
        fig.append_trace(figure["data"][trace], row=r, col=c)

fig.update_layout(height=1500, width=1500, title_text=title)
fig.update_xaxes(tickmode='linear')
# fig.show()
fig.write_html(normalized_onet_bar_chart)

In [12]:
fig = px.line(df, x='year', y='normalized_score', color='cluster', custom_data=['cluster_skill_count', 'cluster'])
fig.update_traces(
    hovertemplate="<br>".join([
        "Cluster: %{customdata[1]}",
        "Year: %{x}",
        "Normalized Score: %{y}",
        "Count: %{customdata[0]}",
    ])
)

fig.update_layout(height=500, width=1500, title_text=title)
fig.update_xaxes(tickmode='linear')
fig.write_html(normalized_onet_line_chart)